# LMA: Levenberg-Marquardt Algorithm

The `LMA` operator implements the Levenberg-Marquardt algorithm, an iterative procedure widely used for solving non-linear least squares problems or for finding roots of non-linear systems of equations. This implementation is designed to be robust, but at the same time provides sensible defaults to facilitate its usage.

## Introduction

The LMA method interpolates between the more aggressive Gauss-Newton algorithm and the more conservative method of gradient descent.

It is an iterative algorithm. Every iteration, the residual and the Jacobian are calculated for a given set of parameters $p$. Then a new guess $p-\Delta p$ is calculated, such that:

$$(J^T W J + \lambda D)\Delta p = J^T W y$$

where $J$ is the Jacobian ($J_{ij}=\partial y_i / \partial p_j$), $W$ is a weight matrix depending on the choice of loss function, $D$ is a diagonal matrix such that $D_{kk} = \max(\epsilon(\lambda), (J^T W J)_{kk})$ with $\epsilon(\lambda)$ the damping floor (dependent on the damping factor), $y$ is the residual, and $\lambda$ is the damping factor. The damping factor determines how much the next guess approximates the prediction of the Gauss-Newton algorithm (lower damping factors) or the gradient descent methods (higher damping factors), in effect defining a trust-region.

For this new guess, the predicted error reduction is calculated as:

$$\Delta s_p = \frac{1}{2}(\Delta p^T J^T W y + \lambda \Delta p^T D \Delta p)$$

If the ratio of the actual error reduction calculated during the next iteration with respect to this prediction is above a defined limit, the current guess is accepted and the damping factor is decreased. Else, the new guess is rejected and the damping factor is increased.

The algorithm finishes once one of these conditions is met, returning the last accepted guess:

* Maximum numbers of iterations reached
* Residual below specified tolerance
* Relative change in parameters or residual below specified tolerance
* Stagnation with maximum damping factor

### Choice of loss function

The standard $L_2$ loss function calculates the loss as the square of the residual. Its corresponding weight matrix is the identity. Moreover, if an individual scaling factor is defined for each residual, this loss function can be used for the solution of *weighted least-squares* problems.

Robust loss functions provide mechanisms to mitigate the effect of outliers in fitting data:

* **Huber** Equivalent to $L_2$ for small residuals and linear for larger residuals
* **Cauchy** Strongly down-weights large outliers
* **L1-Soft** Smooth approximation of $L_1$ loss that behaves like $L_2$ for small residuals
* **Tukey** Redescending M-estimator that completely rejects extreme outliers (but it may lead to convergence issues if scaling is not chosen well)
* **Welsh** Another redescending M-estimator, smoother than Tukey's in its rejection
* **Fair** Less sensitive to large errors than L2, but not redescending
* **Arctan** Limits maximum loss of single residuals

### Normalized damping factor

A particularity of this LMA implementation is the use of a *normalized damping factor*. The damping factor $\lambda$ will oscillate between the specified limits $\lambda_{min}$ and $\lambda_{max}$, starting with an initial value $\lambda_0$. The normalized damping factor is calculated as:

$$\lambda_{norm} = \frac{(\lambda_{max}-\lambda_0)(\lambda-\lambda_{min})}{(\lambda_0-\lambda_{min})(\lambda_{max}-\lambda)}$$

If $\lambda_{norm}$ is 1, the initial damping factor is being used. If it is larger, it means that more damping than indicated was necessary, reaching the maximum $\lambda_{max}$ at infinite; if it is lower, it means that it could be decreased for faster convergence, with the point at 0 corresponding with $\lambda_{min}$.

The usage of normalized damping factors allows to monitor and adjust the effective size of the trust region during successive function calls independently of the actual damping parameters at use.

### Adaptive damping floor

To enhance numerical stability, particularly when dealing with Jacobian matrices $J$ where $J^T J$ may have very small or zero diagonal elements, this LMA implementation employs an adaptive floor for the diagonal elements of the damping scaling matrix $D$. The floor $\epsilon(\lambda)$ is calculated such that it takes the value $\epsilon_0$ (an arbitrarily small number) when $\lambda$ is $\lambda_0$ or lower than $\lambda_0$ (so, $\lambda_{norm}≤1$), and it approaches 1 as $\lambda$ approaches $\lambda_{max}$.

This adaptive mechanism ensures that while a very low floor is used during optimistic (low damping) phases, a more substantial floor (approaching 1) is automatically applied to weak components when high overall damping is required.

## Usage

### ` Jacobian` Operator

    R←{X}f Jacobian Y
    
`Jacobian` is a monadic operator that takes a monadic function `f` as left operand to return an ambivalent function. This derived function returns an estimation of the Jacobian matrix of `f`, using the method of finite differences. The right argument `Y` is the value at which the Jacobian is calculated, and the optional left argument `X` is the perturbation to apply to `Y` in the finite differences method. If `X` is not a given, `⎕CT` is used.

### `LMA` Operator

    R←{X}f LMA Y

`LMA` is a monadic operator, which takes a left operand to return a derived ambivalent function. This derived function allows to minimize a residual function with a known Jacobian using the Levenberg-Marquadt algorithm, given an initial set of parameters. Several configuration options are available, with sensible defaults previously defined.

The left operand `f` must be a configuration namespace or a function. Configuration namespaces may define the following configuration parameters:

* `toli`: Maximum number of iterations (default `1E3`)
* `tols`: Tolerance for the sum of squared residuals (default `⎕CT`)
* `tolr`: Tolerance for relative change, either in the solution or the residual (default `⎕CT`)
* `tolg`: Tolerance for the gain ratio to accept or reject a step (default `1E¯2`)
* `dini`: Initial damping factor for `dnorm=1` (default `1E¯2`)
* `dinc`: Increment of damping factor after rejected solution (default `5`)
* `ddec`: Decrement of damping factor after accepted solution (default `÷dinc`)
* `dmax`: Maximum damping factor (default `÷⎕CT`)
* `dmin`: Minimum damping factor (default `÷dmax`)
* `dp`: Perturbation applied to parameters for numerical estimation of the Jacobian (default `⎕CT`)
* `loss`: Choice of loss function (default `L2`, also `Huber` `Cauchy` `L1Soft` `Tukey` `Welsh` `Fair` `Arctan` or monadic function)
* `scale`: Scale factor passed as left argument to loss function (default for 95% efficiency in robust loss functions)
* `verbose`: If `1`, print `iter ssr rel dnorm p` each iteration (default `0`)

Configuration namespaces may also contain the functions:

* `Callback`: Callback function (default `⊢`)
* `Eval`: Evaluation function

The evaluation function `Eval` must return either the residuals and the Jacobian for the given set of parameters, or only the residuals. Whenever the residual and Jacobian need to be evaluated, the function `Eval` will be called with trial parameters as right argument and left argument `X`, if given (`Eval` will be called monadically if the derived function `f LMA` is called monadically). `Eval` must return either a two elements vector with the residuals in the first element and the Jacobian in the second one, or a vector of residuals, enclosed if they are not simple scalars. If a Jacobian is not returned, a numerical estimation is calculated evaluating the residual function after applying small perturbations to the parameters (as defined by `dp`, which can be a scalar or a vector).

The `Callback` function will be called every iteration before checking convergence, with a solution namespace for the current guess as right argument and `X` as left argument, if given (`Callback` will be called monadically if the derived function `f LMA` is called monadically). Its return value is discarded.

If `f` is a function, the result is equivalent to using as `f` a namespace with an `Eval` fuction `f`
(with default values for the rest of parameters).

`Y` must be a vector.
The first element of `Y`, or `⊂Y` if `1=≡⍵`, contains the initial guess for the parameters.
If the next element of `Y` is a scalar numeric value, it is interpreted as the initial normalized damping factor.
Additional elements of `Y` must be configuration namespaces. The final configuration parameters are obtained
overwriting the parameters in the namespace given as left operand with those given as right argument from right to left. Default values will be used for non-defined parameters and the `Callback` function, but the `Eval` function must be defined by the user either as left operand `f` or as member of a configuration namespace.

The returned value `R` is a solution namespace corresponding to the last accepted solution.
A solution namespace is a configuration namespace with all the parameters used when running the algorithm and the additional elements:

* `iter`: Number of iterations
* `ssr`: Sum of squared residuals
* `rel`: Relative change metric
* `dnorm`: Normalized damping factor
* `p0`: Initial guess
* `p`: Accepted guess

#### Notes

* With the exception of `toli` and `tols`, configuration parameters should be modified only by expert users or in case of convergence problems

* The relative change metric `rel` is the minimum relative change between successive accepted solutions either in the the sum of squared residuals or in the parameters themselves

* In addition to being used for the definition of default values, `⎕CT` is also the baseline for adaptive floor damping


### `LM` Operator

`LM` is a simplified version of `LMA`. It is a dyadic operator from which a dydadic function is derived. Usage:

    R←X f LM g Y

where `f` is a monadic evaluation function, `g` is a monadic function which takes as argument an `iter ssr rel dnorm p` vector and gets called before every convergence check, `Y` is a two elements vector with the initial guess of parameters and normalized damping factor, and `X` is a vector with the configuration parameters `toli tols tolr tolg dini dinc ddec dmax dmin`. The function `f` must return either the residuals (as a simple or nested vector), the residuals and the Jacobian, or the residuals, Jacobian, loss values and weights, for a set of parameters.

The return value `R` is an `iter ssr rel dnorm p` vector.

## Implementation

In [1]:
)clear

clear ws

In [2]:
Jacobian←{⍺←⎕CT ⋄ ⍉↑⍺÷⍨(⍺⍺¨(⊂⍵)+↓↑(-⍳≢⍵)↑¨⍺)-⊆⍺⍺ ⍵}    ⍝ Jacobian matrix of ⍺⍺ at ⍵ applying perturbation ⍺

In [3]:
]dinput
LM←{ti ts tr tg d0 di dd dx dn←⍺ ⋄ p d←⍵ ⋄ ∆←ts⌈⎕CT    ⍝ Levenberg–Marquardt algorithm

    D←((dx-d0)×dn-⍨⊢)÷(d0-dn)×dx-⊢ ⋄ L←1,⍨⊢,(×⍨1∘↑)    ⍝ damping factor normalization(λ) and standard loss(y)
    J←{(1<|≡g)∧1<≢g←⍺⍺ ⍵:g ⋄ (⊃⊆g)(∆ ⍺⍺ Jacobian ⍵)}    ⍝ residual and (estimated if not given) jacobian(p)
    E←⍺⍺{y j l w←L⍣(2=≢g)⊢g←⍺⍺ J ⍵ ⋄ (+/l)⍵ y j w}     ⍝ eval(p): sum(loss), parameters, residual, jacobian
    A←{s p y j w←⍵ ⋄ s p(t+.×j)(y+.×⍨t←w×⍤1⍉j)}        ⍝ accept(EG output): sum(error²), parameters, JtJ, Jty
    
    T←{                                                ⍝ try guess(λ)
        r⊢←0 ⋄ 11::dx⌊⍵×di ⋄ b←1-÷1⌈d⊢←D ⍵             ⍝     bad guess if domain error
        ∆p←jy⌹jj+⍺×⍤1⊢⍵×dj←(⎕CT+b-⎕CT×b)⌈1 1⍉jj        ⍝     change of parameters with adaptive floor
        s0←2÷⍨∆p+.×jy+∆p×⍵×dj                          ⍝     predicted error decrement
        s1←s-⊃g←E⊢q←p-∆p                               ⍝     actual error decrement
        r⊢←(p(-÷⍥(+.×⍨)⊣)q)⌊|s1÷s                      ⍝     relative change in parameters or residuals
        (⎕CT>s0)∧⎕CT<s1:dx⌊⍵×di                        ⍝     if no changing, increase damping
        (⎕CT≤s0)∧tg≥s1÷s0:dx⌊⍵×di                      ⍝     if diverging, increase damping
        dn⌈⍵×dd⊣s p jj jy⊢←A g                         ⍝     accept change, decrease damping
    }
    C←⍵⍵{                                              ⍝ convergence check(λ_prev, λ)
        _←⍺⍺(i⊢←i+1)s r d p                            ⍝     call user function
        (ti<i)∨(dx∧.=⍺ ⍵)∨(ts>s)∨(r>0)∧tr>r            ⍝     iterations, max damping, residual, not changing
    }
    i r←0 ⋄ _←⍵⍵ i s r d p⊣s p jj jy←A E p             ⍝ init
    i s r d p⊣(∘.=⍨⍳≢p)T⍣C{11::dx ⋄ D⍣¯1⊢⍵}d           ⍝ return iterations, ssr, change, norm damping, parameters
}

In [4]:
:Namespace Loss
    l2←1                                               ⍝ standard squared-residual
    L2←{(×⍨⍺×⍵)⍺}
    
    huber←1.345                                        ⍝ ~95% efficiency for normal errors
    Huber←{y←⍺>|⍵ ⋄ Y←y⍨ ⋄ N←~Y                        ⍝ L2 for small residuals, lineal for large ones
        (((⍺÷2)-⍨⍺×|)@N×⍨@Y ⍵)((⍺÷⍨|)@N 1@Y ⍵)
    }
    cauchy←2.385                                       ⍝ ~95% efficiency for normal errors
    Cauchy←{(⍺×(⍺÷2)×⍟1+r)(÷1+r←×⍨⍵÷⍺)}                ⍝ strongly downweights large outliers
    
    softl1←1
    SoftL1←{(⍺×⍺×r-1)(÷r←(1+×⍨⍵÷⍺)*÷2)}                ⍝ L2 for small residuals, L1 for large ones
    
    tukey←4.685                                        ⍝ ~95% efficiency for normal errors
    Tukey←{k←(×⍨⍺)÷6 ⋄ y←⍺>|⍵ ⋄ Y←y⍨ ⋄ N←~Y            ⍝ re-descending M-estimator
        (k@Y(k×1-3*⍨1-⊢)@N⊢r)(0@Y(×⍨1-⊢)@N⊢r←×⍨⍵÷⍺)
    }
    welsh←2.985                                        ⍝ ~95% efficiency for normal errors
    Welsh←{(⍺×(⍺÷2)×1-e)(e←*-×⍨⍵÷⍺)}                   ⍝ smoother re-descending M-estimator
    
    fair←1
    Fair←{(⍺×⍺×r-⍟1+r)(÷1+r←(|⍵)÷⍺)}                   ⍝ no re-descending
    
    arctan←1
    Arctan←{(⍺×(⍺÷2)×¯3○r)(÷1+×⍨r←×⍨⍵÷⍺)}              ⍝ limits maximum loss
:EndNamespace

In [5]:
]dinput
LMA←{⍺←⊢ ⋄ p←⊃w←⊆⍵ ⋄ 0::⎕SIGNAL ⎕EN                    ⍝ pass signals

    n←'d',¨'ini' 'inc' 'dec' 'max' 'min'               ⍝ damping
    n←('isrg',⍨¨⊂'tol'),n                              ⍝ tolerances
    nc←'ddec' 'dmin' ⋄ ns←'scale'                      ⍝ computed defaults
    nr←'iter' 'ssr' 'rel' 'dnorm' 'p'                  ⍝ results
    c←(                                                ⍝ default config
        toli:1000 ⋄ tols:⎕CT ⋄ tolr:⎕CT ⋄ tolg:0.01    ⍝     tolerances
        dini:0.01 ⋄ dinc:5 ⋄ dmax:÷⎕CT ⋄ loss:'L2'     ⍝     damping and loss function
        p0:p ⋄ ∆:⎕CT ⋄ verbose:0                       ⍝     init guess, perturbation, logging
    )
    F←{1((⊂6 0⍕↑),12 ¯5∘⍕¨⍤↓)⍵} ⋄ P←{⎕←F ⍵ ⋄ ⍵}        ⍝ format and print
    D←{⍕'sp',¨':',¨(⊂2 5)⌷F ⍵}                         ⍝ display form
    J←{⍉↑⍺÷⍨(⍺⍺¨(⊂⍵)+↓↑(-⍳≢⍵)↑¨⍺)-⊆⍺⍺ ⍵}               ⍝ estimate Jacobian
    E←{⍺←⊢ ⋄ (1<≡e)∧2=≢e←⍺⍺ ⍵:e ⋄ (⊃⊆e)(⍺ ⍺⍺ J ⍵)}     ⍝ evaluate, and estimate J if needed
    M←(2÷⍨1⊥⊢⌷⍨∘⊂⍋⌷⍨∘⊂∘⌈2÷⍨0 1+≢){1@(0∘=)⍺⍺|⍵-⍺⍺ ⍵}    ⍝ median absolute deviation
    L←{0=c.⎕NC'sigma':∇⍵⊣c.sigma←(M ⍵)÷0.6745          ⍝ loss function (needs stddev estimation)
        3=⎕NC'⍺⍺':⍺⍺ ⍵ ⋄ ~(⊂⍺⍺)∊⍵⍵.⎕NL¯3:⎕SIGNAL 6     ⍝     if no user defined must be in Loss
        c.(scale×sigma)(⍵⍵.⍎⍺⍺)⍵                       ⍝     scaled loss function
    }Loss
    3=⎕NC'⍺⍺':⍺((⍺⍺{⍵.Eval←⍺⍺ ⋄ ⍵}c)∇∇)w               ⍝ ⍺⍺ is Eval function
    (1<≢w)∧~2|⎕DR⊃⌽w:⍺((⎕NS ⍺⍺(⊃⌽w))∇∇)¯1↓w            ⍝ non numeric extra argument is config
    2<≢w:⎕SIGNAL 11                                    ⍝ wrong argument
    
    c.CallBack←⊢ ⋄ c←c ⎕NS ⍺⍺ ⋄ c.dnorm←1⊣⍣(1=≢w)⊃⌽w   ⍝ default callback and actual config
    _←c ⎕VSET⊂(⊂ns),c ⎕VGET⊂ns(Loss ⎕VGET ⎕C c.loss)     ⍝ set scale factor for loss function
    _←c ⎕VSET(↑nc)(c ⎕VGET(↑nc)c.(÷dinc dmax))         ⍝ other computed defaults
    CB←⍺{⍺⍺ c.CallBack c ⎕VSET(↑nr)⍵}P⍣(c.verbose≢0)   ⍝ callback function
    EV←⍺∘c.Eval{y j←⍺ ⍺⍺ E ⍵ ⋄ y j,(c.loss L)y}        ⍝ eval function
    c⊣c.⎕DF D(c ⎕VGET↑n)(c.∆∘EV)LM CB p c.dnorm        ⍝ return namespace
}

### Tests

In [6]:
]dinput
Test←{0::⎕EM ⎕SIGNAL ⎕EN
    t←()
    ⍝ Tests ability to navigate sharp, narrow valleys and handle situations where the Jacobian
    ⍝ may lead to a locally rank-deficient J^T J matrix (eg. zero diagonal elements), requiring
    ⍝ robust damping
    t.Beale←{⎕CT←1e¯10
        B←{
            x y←⍵ ⋄ (1.5 2.25 2.625-x×1-y*⍳3)[(¯1+y)x ⋄ (¯1+y*2)(2×x×y) ⋄ (¯1+y*3)(3×x×y*2)]
        }
        r←3 0.5 CMP(R B #.LMA 1 0.8).p                                ⍝ easy
        r,←3 0.5 CMP(R B #.LMA 1 1).p                                 ⍝ singular Jt J
        r,←3 0.5 CMP(R B #.LMA 0 0).p                                 ⍝ another tricky point
        r,←3 0.5 CMP(R B #.LMA 1 ¯2).p                                ⍝ different quadrant
        r,←(B #.LMA(2 2)(toli:1)).ssr>(R B #.LMA 2 2).ssr             ⍝ closer to another local minimum
        r,←(B #.LMA(¯1 1)(toli:1)).ssr>(R B #.LMA ¯1 1).ssr           ⍝ closer to another local minimum
        r
    }
    ⍝ numerical approximation of the Jacobian
    t.BealeNum←{⎕CT←1e¯10
        B←{
            x y←⍵ ⋄ (1.5 2.25 2.625-x×1-y*⍳3)
        }
        r←3 0.5 CMP(R B #.LMA 1 0.8).p                                ⍝ easy
        r,←3 0.5 CMP(R B #.LMA 1 1).p                                 ⍝ singular Jt J
        r,←3 0.5 CMP(R B #.LMA 0 0).p                                 ⍝ another tricky point
        r,←3 0.5 CMP(R B #.LMA 1 ¯2).p                                ⍝ different quadrant
        r,←(B #.LMA(2 2)(toli:1)).ssr>(R B #.LMA 2 2).ssr             ⍝ closer to another local minimum
        r,←(B #.LMA(¯1 1)(toli:1)).ssr>(R B #.LMA ¯1 1).ssr           ⍝ closer to another local minimum
        r
    }
    ⍝ Assesses capability to follow a long, narrow, curving multi-dimensional valley, testing
    ⍝ the interplay between step length and direction adjustments controlled by the damping factor
    t.Helical←{
        T←{1|1+(12○⍺+0j1×⍵)÷○2} ⋄ M←{|⍺+0j1×⍵}
        H←{
            a b c←⍵ ⋄ m←a M b ⋄ y←(10×c-10×a T b)(10×m-1)c
            y[((50×b)÷○+.×⍨a b)(-(50×a)÷○+.×⍨a b)10 ⋄ (10×a÷m)(10×b÷m)0 ⋄ 0 0 1]
        }
        r←1 0 0 CMP(R H #.LMA¯1 0 0).p                                ⍝ standard starting point
        r,←1 0 0 CMP(R H #.LMA¯1.2 0.1 0.1).p                         ⍝ slightly perturbed
        r,←1 0 0 CMP(R H #.LMA¯0.9 ¯0.05 ¯0.05).p                     ⍝ in other direction
        r,←1 0 0 CMP(R H #.LMA 0.5 ¯0.5 0.5).p                        ⍝ qualitatively different
        r,←1 0 0 CMP(R H #.LMA¯0.5 0.5 ¯0.5).p                        ⍝ another one
        r,←1 0 0 CMP(R H #.LMA¯1 0 10).p                              ⍝ far off in 3rd dimension
        r,←1 0 0 CMP(R H #.LMA¯1 0 ¯10).p                             ⍝ far off in 3rd dimension
        r,←1 0 0 CMP(R H #.LMA 3 4 5).p                               ⍝ away in all components
        r
    }
    ⍝ Verifies efficiency and correctness for a linear system. Converge shoud be very fast
    ⍝ (1 or 2 iterations) with minimal damping, demonstrating Gauss-Newton like behavior
    t.Linear←{
        y←(A←?100 10⍴0)+.×x←⍳10 ⋄ s←R{(y-⍨A+.×⍵)A}#.LMA(?10⍴0)0 ⋄ (1=s.iter),(⍳10)CMP s.p
    }
    ⍝ numerical approximation of the Jacobian
    t.LinearNum←{
        y←(A←?100 5⍴0)+.×x←⍳5 ⋄ s←R{y-⍨A+.×⍵}#.LMA(?5⍴0)0 ⋄ (⍳5)CMP s.p
    }
    ⍝ Evaluates LMA's robustness and ability to converge to a solution when the Jacobian
    ⍝ becomes singular (rank-deficient) at or near the optimum
    t.Powell←{
        P←{
            a b c d←⍵ ⋄ r5 r10←5 10*÷2 ⋄ y←(a+10×b)(r5×c-d)(×⍨b-2×c)(r10××⍨a-d)
            y[1 10 0 0 ⋄ 0 0 r5(-r5) ⋄ 0(2×b-2×c)(-2×b-2×c)0 ⋄ (2×r10×a-d)0 0(-2×r10×a-d)]
        }
        r←(4⍴0)CMP(R P #.LMA(3 ¯1 0 1)(tols:1e¯30)).p                 ⍝ standard starting point
        r,←(4⍴0)CMP(R P #.LMA(0 0 0 0)(tols:1e¯30)).p                 ⍝ solution (convergence at zero step)
        r,←(4⍴0)CMP(R P #.LMA(1 1 1 1)(tols:1e¯30)).p                 ⍝ far from solution
        r
    }
    ⍝ Tests LMA's performance in minimizing a classic non-linear function characterized by
    ⍝ a deep, narrow, banana-shaped valley, requiring effective adaptation of search
    ⍝ direction and step size
    Rosenbrock←{p q←⍵ ⋄ ((10×q-×⍨p),1-p)[(-20×p)10 ⋄ ¯1 0]}
    t.Rosenbrock←{
        r←1 1 CMP(R #.Rosenbrock #.LMA 1.5 1.5).p                     ⍝ close to the solution
        r,←1 1 CMP(R #.Rosenbrock #.LMA 2 1).p                        ⍝ not so close
        r,←1 1 CMP(R #.Rosenbrock #.LMA 0 0).p                        ⍝ outside of parabollic valley
        r,←1 1 CMP(R #.Rosenbrock #.LMA ¯1.2 1).p                     ⍝ further
        r,←1 1 CMP(R #.Rosenbrock #.LMA ¯2 ¯2).p                      ⍝ far and wrongly pointed gradient
        r,←1 1 CMP(R #.Rosenbrock #.LMA 2 2).p                        ⍝ far and wrongly pointed gradient
        r
    }
    ⍝ numerical approximation of the Jacobian
    t.RosenbrockNum←{
        r←1 1 CMP(R⊃⍤#.Rosenbrock #.LMA 1.5 1.5).p                    ⍝ close to the solution
        r,←1 1 CMP(R⊃⍤#.Rosenbrock #.LMA 2 1).p                       ⍝ not so close
        r,←1 1 CMP(R⊃⍤#.Rosenbrock #.LMA 0 0).p                       ⍝ outside of parabollic valley
        r,←1 1 CMP(R⊃⍤#.Rosenbrock #.LMA ¯1.2 1).p                    ⍝ further
        r,←1 1 CMP(R⊃⍤#.Rosenbrock #.LMA ¯2 ¯2).p                     ⍝ far and wrongly pointed gradient
        r,←1 1 CMP(R⊃⍤#.Rosenbrock #.LMA 2 2).p                       ⍝ far and wrongly pointed gradient
        r
    }
    ⍝ Check that algorithm finishes on termination conditions
    t.Terminate←{
        r←(R #.Rosenbrock #.LMA(0 0)(tols:0 ⋄ tolr:0)).(iter>toli)              ⍝ number of iterations
        r,←(R #.Rosenbrock #.LMA(0 0)(tols:0)).(rel<tolr)                       ⍝ relative change
        r,←(R #.Rosenbrock #.LMA(0 0)(tols:0 ⋄ dmax:1e6)).((dnorm>1e6)∧rel=0)   ⍝ maximum damping
        r
    }
    ⍺←1e¯6 ⋄ tol←⍺ ⋄ CMP←{tol>|⍺-⍵} ⋄ R←{⎕←(10↑''),⍵.((6 0⍕iter),(12 ¯5⍕ssr),p) ⋄ ⍵}
    (t⎕NS'tol' 'CMP' 'R'){⎕←⍵ ⋄ 0∊⍺⍎⍵,'⍬':('TEST FAILED: ',⍵)⎕SIGNAL 8 ⋄ _←0}¨(↓t.⎕NL 3)⊣⍣(⍵≡'*')⊆⍵
}

In [7]:
Test'*'

Beale 
 6 9.1911E¯22 3 0.4999999999
 8 9.5670E¯15 2.999999563 0.4999998956
 32 1.6285E¯17 2.999999982 0.4999999953
 8 2.7181E¯17 3 0.5
 780 2.2422E1 ¯15356.14165 1.000064535
 908 1.3975E¯1 ¯15369.82063 1.000064478
BealeNum 
 7 5.8013E¯17 2.999999973 0.4999999969
 10 1.2470E¯17 2.999999999 0.5000000011
 34 1.0296E¯17 2.999999993 0.4999999992
 10 1.2292E¯15 3.000000002 0.4999999994
 79 2.7909E1 ¯13.39491064 1.069653981
 74 1.7473E¯1 ¯13.07858891 1.071139786
Helical 
 10 2.6199E¯19 1 2.177312345E¯10 3.448140009E¯10
 10 9.8160E¯19 1 2.170623215E¯10 3.43759102E¯10
 8 8.9944E¯19 1 4.309360933E¯10 6.82517468E¯10
 14 1.2103E¯18 1 1.364812195E¯10 2.161410116E¯10
 12 2.0384E¯19 1 7.906934901E¯11 1.252178359E¯10
 24 6.0269E¯18 1 1.0443077E¯10 1.653826613E¯10
 22 7.8009E¯19 1 3.757169247E¯11 5.949970095E¯11
 10 1.8057E¯18 1 1.201271532E¯10 1.902399671E¯10
Linear 
 1 4.8419E¯23 1 2 3 4 5 6 7 8 9 10
LinearNum 
 7 3.8950E¯15 0.9999999995 2.000000008 2.999999998 3.999999998 5
Powell 
 833 9.9686E¯31 9.199960843E¯9 ¯9.199960843E¯10 5.964005809E¯9 5.964005809E¯9
 1 0.0000E0 0 0 0 0
 68 9.8111E¯31 ¯2.615562126E¯8 2.615562126E¯9 ¯1.676709063E¯8 ¯1.676709063E¯8
Rosenbrock 
 9 5.1581E¯20 1 1
 13 9.2491E¯19 1 1
 32 2.8174E¯15 0.999999929 0.999999857
 29 3.0252E¯20 1 0.9999999999
 19 2.9277E¯18 1 0.9999999999
 15 4.0858E¯19 1 1
RosenbrockNum
 9 2.0513E¯19 1 1
 13 5.1828E¯18 1 1
 32 1.3489E¯15 0.9999999592 0.9999999155
 29 6.2797E¯14 0.9999999579 0.9999999129
 19 1.3570E¯18 1 1
 15 3.7632E¯17 1 1
Terminate 
 1001 0.0000E0 1 1
 34 7.8299E¯30 1 1
 13 5.4951E¯1 0 0

### Export

In [8]:
] _←link.export -overwrite # APLSource

## Examples

In [9]:
NOISY←{⍵+⍺×0.5-⍨?0⍴⍨≢⍵}

### Exponential decay

In [10]:
ExpDec←{A k C←⍺ ⋄ C+A×*-k×⍵} ⋄ ExpDecEv←{x y←⍺ ⋄ A k C←⍵ ⋄ xe←A×x×e←*-k×x ⋄ (y-⍵ ExpDec x)(⍉(-e)⍪xe⍪⍉⍪-=⍨e)}
p←10 0.5 1 ⋄ y←0.25 NOISY y0←p ExpDec x←(⍳10)-1 ⋄ ⊂5⍕↑x y(p ExpDec x)(⊃x y ExpDecEv p)
⊢s←x y∘ExpDecEv LMA(5 0.1 0.5)(verbose:1)

┌─────────────────────────────────────────────────────────────────────────────────────┐
│ 0.00000 1.00000 2.00000 3.00000 4.00000 5.00000 6.00000 7.00000 8.00000 9.00000│
│ 11.06561 7.12200 4.65023 3.34979 2.38845 1.73589 1.51637 1.18956 1.10165 1.15917│
│ 11.00000 7.06531 4.67879 3.23130 2.35335 1.82085 1.49787 1.30197 1.18316 1.11109│
│ 0.06561 0.05670 ¯0.02856 0.11849 0.03510 ¯0.08496 0.01850 ¯0.11241 ¯0.08150 0.04808│
└─────────────────────────────────────────────────────────────────────────────────────┘

0 1.6301E1 0.0000E0 1.0000E0 5.0000E0 1.0000E¯1 5.0000E¯1 
 1 7.7765E0 2.7856E¯1 1.0000E0 6.5880E0 3.8366E¯1 2.6058E0 
 2 8.1250E¯2 2.6688E¯1 2.0000E¯1 9.9276E0 5.4385E¯1 1.1040E0 
 3 1.3635E¯2 4.9059E¯4 4.0000E¯2 1.0092E1 4.9076E¯1 9.6501E¯1 
 4 1.2167E¯2 4.8993E¯6 8.0000E¯3 1.0109E1 4.9336E¯1 9.5072E¯1 
 5 1.2167E¯2 5.7512E¯11 1.6000E¯3 1.0109E1 4.9336E¯1 9.5068E¯1 
 6 1.2167E¯2 2.7307E¯19 3.2000E¯4 1.0109E1 4.9336E¯1 9.5068E¯1 
 s: 1.2167E¯2 p: 1.0109E1 4.9336E¯1 9.5068E¯1

### Numerical estimation of Jacobian

In [11]:
+.×⍨,(1e¯8(y-ExpDec∘x)Jacobian p)-⊃⌽x y ExpDecEv p

5.069317867E¯14

### Direct use of `LM` operator

In [12]:
cfg←1e6 ⎕CT ⎕CT 1e¯2 1e¯2 5(÷5)(÷⎕CT)⎕CT         ⍝ ti ts tr tg d0 di dd dx dn
cfg((1,⍨⊢,(×⍨1∘↑))x y∘ExpDecEv)LM⊢(5 0.1 0.5)1   ⍝ left operand returns everything
cfg(x y∘ExpDecEv)LM⊢(5 0.1 0.5)1                 ⍝ left operand returns residual and jacobian
cfg(y-ExpDec∘x)LM⊢(5 0.1 0.5)1                   ⍝ left operand only returns residuals

┌─┬─────────────┬───────────────┬──────────────┬─────────────────────────────────────┐
│6│0.03901635339│2.730736981E¯19│0.000319999999│10.10907949 0.4933561892 0.9506752553│
└─┴─────────────┴───────────────┴──────────────┴─────────────────────────────────────┘

┌─┬─────────────┬───────────────┬──────────────┬─────────────────────────────────────┐
│6│0.03901635339│2.730736981E¯19│0.000319999999│10.10907949 0.4933561892 0.9506752553│
└─┴─────────────┴───────────────┴──────────────┴─────────────────────────────────────┘

┌──┬─────────────┬───────────────┬─────┬─────────────────────────────────────┐
│30│0.03901661062│5.280092878E¯15│78125│10.10906157 0.4934182298 0.9509600169│
└──┴─────────────┴───────────────┴─────┴─────────────────────────────────────┘

In [13]:
⊢s←x y0∘ExpDecEv LMA(10 0.5 1)(verbose:1 ⋄ loss:'L2')                   ⍝ exact
⊢s←x y0∘ExpDecEv LMA(10 0.5 1)(verbose:1 ⋄ loss:'Huber')                ⍝ exact
⊢s←x y∘ExpDecEv LMA(5 0.1 0.5)(verbose:1 ⋄ loss:'Huber')
⊢s←x y∘ExpDecEv LMA(5 0.1 0.5)(verbose:1 ⋄ loss:'Huber' ⋄ scale:10)
⊢s←x y∘ExpDecEv LMA(5 0.1 0.5)(verbose:1 ⋄ loss:'Cauchy')
⊢s←x y∘ExpDecEv LMA(5 0.1 0.5)(loss:'SoftL1')
⊢s←x y∘ExpDecEv LMA(5 0.1 0.5)(loss:'Tukey' ⋄ scale:1e¯1)               ⍝ needs scaling to converge
⊢s←x y∘ExpDecEv LMA(5 0.1 0.5)(loss:'Welsh')
⊢s←x y∘ExpDecEv LMA(5 0.1 0.5)(loss:'Fair')
⊢s←x y∘ExpDecEv LMA(5 0.1 0.5)(loss:'Arctan')

0 0.0000E0 0.0000E0 1.0000E0 1.0000E1 5.0000E¯1 1.0000E0 
 1 0.0000E0 0.0000E0 1.0000E0 1.0000E1 5.0000E¯1 1.0000E0 
 s: 0.0000E0 p: 1.0000E1 5.0000E¯1 1.0000E0

0 0.0000E0 0.0000E0 1.0000E0 1.0000E1 5.0000E¯1 1.0000E0 
 1 0.0000E0 0.0000E0 1.0000E0 1.0000E1 5.0000E¯1 1.0000E0 
 s: 0.0000E0 p: 1.0000E1 5.0000E¯1 1.0000E0

0 1.0381E1 0.0000E0 1.0000E0 5.0000E0 1.0000E¯1 5.0000E¯1 
 1 1.0381E1 9.6692E¯2 1.0000E0 5.0000E0 1.0000E¯1 5.0000E¯1 
 2 4.0162E0 4.6668E¯1 5.0000E0 8.2005E0 3.7006E¯1 1.7133E0 
 3 1.5276E¯1 4.4432E¯2 1.0000E0 9.8660E0 5.0960E¯1 1.1380E0 
 4 3.9165E¯2 9.2283E¯4 2.0000E¯1 1.0105E1 4.9311E¯1 9.5433E¯1 
 5 3.9016E¯2 2.6764E¯7 4.0000E¯2 1.0109E1 4.9336E¯1 9.5068E¯1 
 6 3.9016E¯2 8.3670E¯13 8.0000E¯3 1.0109E1 4.9336E¯1 9.5068E¯1 
 7 3.9016E¯2 1.2694E¯19 1.6000E¯3 1.0109E1 4.9336E¯1 9.5068E¯1 
 s: 3.9016E¯2 p: 1.0109E1 4.9336E¯1 9.5068E¯1

0 5.2273E1 0.0000E0 1.0000E0 5.0000E0 1.0000E¯1 5.0000E¯1 
 1 2.4938E1 2.7856E¯1 1.0000E0 6.5880E0 3.8366E¯1 2.6058E0 
 2 2.6055E¯1 2.6688E¯1 2.0000E¯1 9.9276E0 5.4385E¯1 1.1040E0 
 3 4.3725E¯2 4.9059E¯4 4.0000E¯2 1.0092E1 4.9076E¯1 9.6501E¯1 
 4 3.9016E¯2 4.8993E¯6 8.0000E¯3 1.0109E1 4.9336E¯1 9.5072E¯1 
 5 3.9016E¯2 5.7512E¯11 1.6000E¯3 1.0109E1 4.9336E¯1 9.5068E¯1 
 6 3.9016E¯2 2.7307E¯19 3.2000E¯4 1.0109E1 4.9336E¯1 9.5068E¯1 
 s: 3.9016E¯2 p: 1.0109E1 4.9336E¯1 9.5068E¯1

0 8.8778E0 0.0000E0 1.0000E0 5.0000E0 1.0000E¯1 5.0000E¯1 
 1 4.8928E0 7.4556E¯2 1.0000E0 5.9742E0 2.8697E¯1 1.4483E0 
 2 8.0956E¯1 2.5044E¯1 2.0000E¯1 9.0350E0 5.7371E¯1 1.6329E0 
 3 5.8604E¯2 1.7586E¯2 4.0000E¯2 1.0065E1 4.7931E¯1 9.8667E¯1 
 4 1.9487E¯2 3.0439E¯5 8.0000E¯3 1.0107E1 4.9323E¯1 9.5226E¯1 
 5 1.9472E¯2 6.1405E¯8 1.6000E¯3 1.0109E1 4.9336E¯1 9.5067E¯1 
 6 1.9472E¯2 1.9213E¯12 3.2000E¯4 1.0109E1 4.9336E¯1 9.5066E¯1 
 7 1.9472E¯2 5.6656E¯17 6.4000E¯5 1.0109E1 4.9336E¯1 9.5066E¯1 
 s: 1.9472E¯2 p: 1.0109E1 4.9336E¯1 9.5066E¯1

s: 1.9406E¯2 p: 1.0109E1 4.9337E¯1 9.5064E¯1

s: 1.0624E¯2 p: 1.0081E1 4.9080E¯1 9.5102E¯1

s: 1.9485E¯2 p: 1.0109E1 4.9336E¯1 9.5067E¯1

s: 1.7845E¯2 p: 1.0110E1 4.9342E¯1 9.5046E¯1

s: 1.9505E¯2 p: 1.0109E1 4.9336E¯1 9.5067E¯1

In [14]:
⊢ed←x y ExpDecEv LMA⊂5 0.1 0.5
x y(ed LMA)ed.p0

s: 1.2167E¯2 p: 1.0109E1 4.9336E¯1 9.5068E¯1

s: 1.2167E¯2 p: 1.0109E1 4.9336E¯1 9.5068E¯1

In [15]:
x y(ed LMA)ed.p0(verbose:1)

0 1.6301E1 0.0000E0 1.0000E0 5.0000E0 1.0000E¯1 5.0000E¯1 
 1 7.7765E0 2.7856E¯1 1.0000E0 6.5880E0 3.8366E¯1 2.6058E0 
 2 8.1250E¯2 2.6688E¯1 2.0000E¯1 9.9276E0 5.4385E¯1 1.1040E0 
 3 1.3635E¯2 4.9059E¯4 4.0000E¯2 1.0092E1 4.9076E¯1 9.6501E¯1 
 4 1.2167E¯2 4.8993E¯6 8.0000E¯3 1.0109E1 4.9336E¯1 9.5072E¯1 
 5 1.2167E¯2 5.7512E¯11 1.6000E¯3 1.0109E1 4.9336E¯1 9.5068E¯1 
 6 1.2167E¯2 2.7307E¯19 3.2000E¯4 1.0109E1 4.9336E¯1 9.5068E¯1 
 s: 1.2167E¯2 p: 1.0109E1 4.9336E¯1 9.5068E¯1

In [16]:
⊢s←x y{x y←⍺ ⋄ y-⍵ ExpDec x}LMA(5 0.1 0.5)

s: 1.2168E¯2 p: 1.0108E1 4.9355E¯1 9.5198E¯1

### Linear system

In [17]:
Linear←{a b←⍺ ⋄ a+b×⍵} ⋄ LinearEv←{(y-⍵ Linear x)(⍉↑-(=⍨x)x)}
p←0.1 2 ⋄ y←1e¯3 NOISY p Linear x←(⍳10)-1 ⋄ ⊂5⍕↑x y(p Linear x)(⊃LinearEv p)
LinearEv LMA(0 0)(dinc:1e3 ⋄ verbose:1)

┌─────────────────────────────────────────────────────────────────────────────────────────┐
│ 0.00000 1.00000 2.00000 3.00000 4.00000 5.00000 6.00000 7.00000 8.00000 9.00000│
│ 0.09957 2.09979 4.09995 6.10012 8.09959 10.10025 12.09993 14.10048 16.09971 18.09978│
│ 0.10000 2.10000 4.10000 6.10000 8.10000 10.10000 12.10000 14.10000 16.10000 18.10000│
│ ¯0.00043 ¯0.00021 ¯0.00005 0.00012 ¯0.00041 0.00025 ¯0.00007 0.00048 ¯0.00029 ¯0.00022│
└─────────────────────────────────────────────────────────────────────────────────────────┘

0 6.3645E4 0.0000E0 1.0000E0 0.0000E0 0.0000E0 
 1 6.3645E4 0.0000E0 1.0000E0 0.0000E0 0.0000E0 
 2 6.3645E4 0.0000E0 1.0000E3 0.0000E0 0.0000E0 
 3 6.3645E4 0.0000E0 1.0000E6 0.0000E0 0.0000E0 
 4 6.3645E4 0.0000E0 1.0000E9 0.0000E0 0.0000E0 
 5 6.3645E4 3.4300E¯10 1.0001E12 9.0999E¯10 2.0158E¯10 
 6 6.3645E4 3.4300E¯7 1.0000E9 9.1090E¯7 2.0178E¯7 
 7 6.3623E4 3.4291E¯4 1.0000E6 9.1072E¯4 2.0175E¯4 
 8 4.6600E4 2.6756E¯1 1.0000E3 7.5750E¯1 1.7256E¯1 
 9 1.7989E1 9.9961E¯1 1.0000E0 3.8691E¯1 1.9370E0 
 10 3.9941E¯5 2.2138E¯2 1.0000E¯3 9.9827E¯2 2.0000E0 
 11 3.9873E¯5 1.0034E¯10 1.0000E¯6 9.9807E¯2 2.0000E0 
 12 3.9873E¯5 4.1130E¯25 9.9900E¯10 9.9807E¯2 2.0000E0 
 s: 3.9873E¯5 p: 9.9807E¯2 2.0000E0

### Gauss function

In [18]:
Gauss←{a s m c←⍺ ⋄ c+a×m s X ⍵} ⋄ X←{m s←⍺ ⋄ *-(×⍨⍵-m)÷(2××⍨s)}
GaussEv←{x y←⍺ ⋄ a s m c←⍵ ⋄ x2←(x1←(xe←m s X x)×a×(x-m)÷×⍨s)×(x-m)÷s ⋄ (y-⍵ Gauss x)(⍉↑(-xe)x1(-x2)(-=⍨xe))}
p←10 5 1.5 2 ⋄ y←0 NOISY p Gauss x←(⍳10)-1 ⋄ ⊂5⍕↑x y(p Gauss x)(⊃x y GaussEv p)
(x y GaussEv LMA(8 4 1 1)(dini:1e¯3 ⋄ tols:1e¯6)).(iter ssr rel dnorm p)
(x y GaussEv LMA(8 4 1 1)1e¯1(tols:1e¯6)).(iter ssr rel dnorm p)
(x y GaussEv LMA(8 4.5 1.2 1.5)(tolr:1e¯3)).(iter ssr rel dnorm p)

┌─────────────────────────────────────────────────────────────────────────────────────┐
│ 0.00000 1.00000 2.00000 3.00000 4.00000 5.00000 6.00000 7.00000 8.00000 9.00000│
│ 11.55997 11.95012 11.95012 11.55997 10.82497 9.82705 8.66977 7.46074 6.29557 5.24652│
│ 11.55997 11.95012 11.95012 11.55997 10.82497 9.82705 8.66977 7.46074 6.29557 5.24652│
│ 0.00000 0.00000 0.00000 0.00000 0.00000 0.00000 0.00000 0.00000 0.00000 0.00000│
└─────────────────────────────────────────────────────────────────────────────────────┘

┌──┬───────────┬───────────────┬──────────┬───────────────────────────────────────────────┐
│18│1.033325579│1.377060143E¯15│1220703140│8.921655369 3.871347834 2.623807742 3.046908918│
└──┴───────────┴───────────────┴──────────┴───────────────────────────────────────────────┘

┌──┬───────────┬───────────────┬─────────┬───────────────────────────────────────────────┐
│18│1.033325579│1.377060143E¯15│122070314│8.921655369 3.871347834 2.623807742 3.046908918│
└──┴───────────┴───────────────┴─────────┴───────────────────────────────────────────────┘

┌─┬─────────────┬───────────────┬──┬───────────────────────────────────────────────┐
│7│0.05676122587│0.0007211535235│25│9.471886955 4.249297788 2.168664717 2.526269831│
└─┴─────────────┴───────────────┴──┴───────────────────────────────────────────────┘